<a href="https://colab.research.google.com/github/Louis28240/AutoRetranscription/blob/main/RetranscriptionAutoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

import torch
print(f"\nGPU disponible : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Modèle de GPU : {torch.cuda.get_device_name(0)}")

In [ ]:
!pip install openai-whisper
!pip install ipywidgets
!pip install ffmpeg-python
!apt-get update && apt-get install -y ffmpeg

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import whisper
from google.colab import files
import os
import time
import threading
from datetime import datetime, timedelta
import re
import torch
import numpy as np
import tqdm

class WhisperGUI:
    def __init__(self):
        if not torch.cuda.is_available():
            display(HTML("""
            <div style='background-color: #ffebee; padding: 10px; border-radius: 5px; margin-bottom: 10px;'>
                <h3 style='color: #c62828;'>⚠️ Attention : GPU non détecté !</h3>
                <p>Pour de meilleures performances, activez le GPU dans Runtime > Change runtime type</p>
            </div>
            """))

        display(HTML("""
        <style>
        .widget-label { font-size: 16px !important; }
        .custom-header {
            background-color: #4CAF50;
            color: white;
            padding: 20px;
            border-radius: 10px;
            margin-bottom: 20px;
        }
        .transcription-box {
            background-color: white;
            color: #000000;
            font-size: 16px;
            line-height: 1.6;
            padding: 20px;
            border-radius: 5px;
            margin: 10px 0;
            height: 300px;
            overflow-y: auto;
            border: 1px solid #ccc;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            font-weight: 500;
            white-space: pre-wrap;
            word-wrap: break-word;
        }
        .download-message {
            background-color: #e3f2fd;
            color: #1565c0;
            padding: 10px;
            border-radius: 5px;
            margin: 10px 0;
            border: 1px solid #90caf9;
            font-weight: bold;
        }
        </style>
        """))

        gpu_info = f"🚀 GPU: {torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "💻 Mode CPU (plus lent)"
        display(HTML(f"""
        <div class='custom-header'>
            <h1>🎙️ Transcription Audio avec Whisper</h1>
            <p>Un outil simple pour transcrire vos fichiers audio en texte</p>
            <p><small>{gpu_info}</small></p>
        </div>
        """))

        self.model_dropdown = widgets.Dropdown(
            options=[
                ('Rapide (Qualité basique)', 'tiny'),
                ('Standard (Bon compromis)', 'base'),
                ('Haute qualité', 'small'),
                ('Très haute qualité', 'medium'),
                ('Qualité maximale', 'large')
            ],
            value='base',
            description='Qualité:',
            style={'description_width': 'initial'},
            layout={'width': '50%'}
        )

        self.upload_button = widgets.Button(
            description='Choisir un fichier audio',
            button_style='success',
            icon='upload'
        )

        self.status_label = widgets.HTML(
            value='📊 Statut: En attente du fichier...'
        )

        self.transcription_output = widgets.HTML(
            value='<div class="transcription-box">La transcription apparaîtra ici...</div>'
        )

        self.download_message = widgets.HTML()

        self.output = widgets.Output()

        self.create_layout()

        self.upload_button.on_click(self.handle_upload)

    def create_layout(self):
        display(widgets.VBox([
            widgets.HTML('<h3>1. Choisissez la qualité de transcription</h3>'),
            self.model_dropdown,
            widgets.HTML('<br><h3>2. Sélectionnez votre fichier audio</h3>'),
            self.upload_button,
            widgets.HTML('<br><h3>3. Statut</h3>'),
            self.status_label,
            self.download_message,
            widgets.HTML('<br><h3>4. Transcription</h3>'),
            self.transcription_output,
            self.output
        ]))

    def update_transcription(self, text):
        """Mise à jour de la transcription"""
        self.transcription_output.value = f'<div class="transcription-box">{text}</div>'

    def transcribe_realtime(self, model, audio_path):
        """Transcription en temps réel"""
        try:

            audio = whisper.load_audio(audio_path)
            segment_length = 30 * 16000
            segments = [audio[i:i+segment_length] for i in range(0, len(audio), segment_length)]

            full_text = ""
            for i, segment in enumerate(segments):

                result = model.transcribe(segment, language="fr")
                segment_text = result["text"].strip()

                full_text += segment_text + " "

                self.update_transcription(full_text)

            return full_text

        except Exception as e:
            print(f"Erreur lors de la transcription : {str(e)}")
            return None

    def handle_upload(self, change):
        with self.output:
            clear_output()
            self.download_message.value = ''
            print("⏳ Upload du fichier en cours...")

            try:
                uploaded = files.upload()
                start_time = time.time()

                for filename in uploaded.keys():
                    self.status_label.value = f"📊 Statut: Préparation du fichier {filename}..."

                    self.status_label.value = "📊 Statut: Chargement du modèle Whisper..."
                    model = whisper.load_model(self.model_dropdown.value)

                    self.status_label.value = "📊 Statut: Transcription en cours..."

                    final_text = self.transcribe_realtime(model, filename)

                    if final_text:
                        output_file = f"{os.path.splitext(filename)[0]}_transcription.txt"
                        with open(output_file, 'w', encoding='utf-8') as f:
                            f.write(final_text)

                        self.status_label.value = "✅ Transcription terminée !"

                        self.download_message.value = f'''
                        <div class="download-message">
                            💾 Téléchargement automatique du fichier texte en cours...<br>
                            Si le téléchargement ne démarre pas automatiquement, vérifiez les paramètres de votre navigateur.
                        </div>'''

                        duration = time.time() - start_time
                        print(f"\n✅ Transcription terminée en {int(duration)} secondes !")

                        files.download(output_file)

            except Exception as e:
                self.status_label.value = f"❌ Erreur: {str(e)}"
                print(f"❌ Erreur : {str(e)}")

# Lancement de l'interface
WhisperGUI()